<a href="https://colab.research.google.com/github/LeoVal1/ECE_505_Project-Optimization-/blob/main/ECE_505_Project_(Optimization).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import sys


In [23]:
class unconstrained_optimizer:
    def __init__(self):
        return
    
    
    #function is the mathematical function that needs to be optimized
    #gradient is the gadient of the mathematical function.
    #line_search is method of finding alpha
    def Steepest_Gradient_Descent_Method(self,function,gradient,line_search,
                                         linear_first_derivative,condition_check,
                                         epsilon,Iter_max,seed_x,linear_second_derivative=None,condition_param=0.5,seed_alpha=0.5,in_param=None,gamma_param=None):
        #List to store values to be printed at each iteration
        grad_value=[]
        x_value=[]
        alpha_value=[]
        next_x=np.array(seed_x)
        current_alpha_value=seed_alpha
        for i in range(Iter_max):
            #print("Iteration:")
            #print(i)
            current_x=next_x
            current_grad_value=gradient(current_x,in_param,gamma_param)
            x_value.append(current_x)
            grad_value.append(current_grad_value)
            f_of_x=function(current_x,in_param,gamma_param)
            criteria=np.linalg.norm(current_grad_value,2)/(1+np.absolute(f_of_x))
            
            if criteria <= epsilon:
                print("Criteria of epsilon met at i:")
                print(i)
                return[x_value,grad_value,alpha_value,criteria]
            if i==25 and condition_param<1e-3:
                condition_param=0.8
                alpha=0.9
            elif i==50 and condition_param<1e-3:
                condition_param=0.8
                alpha=0.9
            elif i==100 and condition_param<1e-2:
                condition_param=0.8
                alpha=0.9
            elif i==200 and condition_param<1e-1:
                condition_param=0.8
                alpha=0.9
            elif i==300:
                alpha=1.5
            elif i==500 :
                condition_param=0.8
                alpha=1.5
            elif i==700:
                alpha=1.5
            
            p=-current_grad_value
            current_alpha_value=line_search(current_x,p,current_alpha_value,function,current_grad_value, 
                                            linear_first_derivative,linear_second_derivative,condition_check,
                                            (condition_param),in_param,gamma_param)
            alpha_value.append(current_alpha_value)
            next_x=current_x+(current_alpha_value*p)
        
        print("Maximum iteration reached.Criteria of epsilon not met")
        current_x=next_x
        current_grad_value=gradient(current_x,in_param,gamma_param)
        x_value.append(current_x)
        grad_value.append(current_grad_value)
        criteria=np.linalg.norm(current_grad_value,2)/(1+np.linalg.norm(current_grad_value,1))        
        return[x_value,grad_value,alpha_value,criteria]
    
    
    def Newtons_Method(self,function,gradient_function,hessian_function,epsilon,Iter_max,seed_x,in_param=None,gamma_param=None):
        x_value=[]
        grad_value=[]
        direction_value=[]
        next_x=np.array(seed_x)
        for i in range(Iter_max):
            #print("Iteration :")
            #print(i)
            current_x_value=next_x
            current_grad_value=gradient_function(current_x_value,in_param,gamma_param)
            f_of_x=function(current_x_value,in_param,gamma_param)
            criteria=np.linalg.norm(current_grad_value,2)/(1+np.absolute(f_of_x))
            x_value.append(current_x_value)
            grad_value.append(current_grad_value)
            if criteria <= epsilon:
                print("Criteria of epsilon met at i:")
                print(i)
                return[x_value,grad_value,direction_value,criteria]
            current_hessian_value=hessian_function(current_x_value,in_param,gamma_param)
            if np.linalg.det(current_hessian_value)<1e-5:
                print("Warning: Hessian Matrix is Singluar for the current estimate of X!!!!!")
            
            current_direction_value=np.linalg.lstsq(current_hessian_value,current_grad_value)[0]
            direction_value.append(current_direction_value)
            current_hessian_inv=np.linalg.pinv(current_hessian_value)
            next_x=current_x_value-current_hessian_inv.dot(current_direction_value)
            
        print("Maximum iteration reached.Criteria of epsilon not met")
        current_x_value=next_x
        current_grad_value=gradient_function(current_x_value,in_param,gamma_param)
        criteria=np.linalg.norm(current_grad_value,2)/(1+np.linalg.norm(current_grad_value,1))
        x_value.append(current_x_value)
        grad_value.append(current_grad_value)
        return[x_value,grad_value,direction_value,criteria]
    
    def BFGS_Quasi_Newton_Method(self,function,gradient_function,line_search,
                                 linear_first_derivative,condition_check,
                                 epsilon,Iter_max,seed_Hessian,seed_x,
                                 linear_second_derivative=None,condition_param=0.5,seed_alpha=0.5,in_param=None,gamma_param=None):
        x_value=[]
        grad_value=[]
        direction_value=[]
        next_x=np.matrix(seed_x).T
        next_B=np.matrix(seed_Hessian)
        next_gradient=np.matrix(gradient_function(next_x,in_param,gamma_param)).T
        alpha=seed_alpha
        for i in range(Iter_max):
            current_x_value=next_x
            current_B=next_B
            current_grad_value=next_gradient
            x_value.append(current_x_value)
            grad_value.append(current_grad_value)
            f_of_x=function(current_x_value,in_param,gamma_param)
            criteria=np.linalg.norm(current_grad_value,2)/(1+np.absolute(f_of_x))
            if criteria <= epsilon:
                print("Criteria of epsilon met at i:")
                print(i)
                return[x_value,grad_value,direction_value,criteria]
            if np.linalg.det(current_B)<1e-5:
                print("Warning:B matrix is Singluar for the current estimate of X!!!!!")
            
            current_direction_value=np.linalg.lstsq(np.squeeze(np.asarray(current_B)),-1*np.squeeze(np.asarray(current_grad_value)))[0]
            current_direction_value=np.matrix(current_direction_value).T
            direction_value.append(current_direction_value)
            alpha=line_search(current_x_value,current_direction_value,alpha,function,current_grad_value, 
                                            linear_first_derivative,linear_second_derivative,condition_check,
                                            condition_param,in_param,gamma_param)
            next_x=current_x_value+(alpha*current_direction_value)
            next_gradient=np.matrix(gradient_function(next_x,in_param,gamma_param)).T
            s_k=next_x-current_x_value
            y_k=next_gradient-current_grad_value
            next_B=current_B-((((current_B*s_k)*(current_B*s_k).T)/(s_k.T*current_B*s_k)))+((y_k*y_k.T)/(y_k.T*s_k))
            
            
        
        print("Maximum iteration reached.Criteria of epsilon not met")
        current_x_value=next_x
        current_grad_value=gradient_function(current_x_value,in_param,gamma_param)
        criteria=np.linalg.norm(current_grad_value,2)/(1+np.linalg.norm(current_grad_value,1))
        x_value.append(current_x_value)
        grad_value.append(current_grad_value)
        return[x_value,grad_value,direction_value,criteria]
     

    def Conjugate_Gradient_Descent_Method(self,function,gradient_function,hessian_function,
                                          epsilon,Iter_max,seed_x,in_param=None,gamma_param=None):
        x_value=[]
        grad_value=[]
        direction_value=[]
        next_x=np.matrix(seed_x).T
        next_gradient=np.matrix(gradient_function(next_x,in_param,gamma_param)).T
        next_direction=-1*next_gradient
        for i in range(Iter_max):
            current_x=next_x
            current_gradient=next_gradient
            current_direction=next_direction
            x_value.append(current_x)
            grad_value.append(current_gradient)
            direction_value.append(current_direction)
            f_of_x=function(current_x,in_param,gamma_param)
            criteria=np.linalg.norm(current_gradient,2)/(1+np.absolute(f_of_x))
            
            if criteria <= epsilon:
                print("Criteria of epsilon met at i:")
                print(i)
                return[x_value,grad_value,direction_value,criteria]
           
            alpha=-1*np.asscalar((current_direction.T*current_gradient)/(current_direction.T*np.matrix(hessian_function(current_x,in_param,gamma_param))*current_direction))
            next_x=current_x+alpha*current_direction
            next_gradient=np.matrix(gradient_function(next_x,in_param,gamma_param)).T
            beta=np.asscalar((current_direction.T*next_gradient)/(current_direction.T*np.matrix(hessian_function(next_x,in_param,gamma_param))*current_direction))
            next_direction=-1*next_gradient+(beta*current_direction)
        
        print("Maximum iteration reached.Criteria of epsilon not met")
        current_x=next_x
        current_gradient=next_gradient
        current_direction=next_direction
        criteria=np.linalg.norm(current_gradient,2)/(1+np.linalg.norm(current_gradient,1))
        x_value.append(current_x)
        grad_value.append(current_gradient)
        direction_value.append(current_direction)
        return[x_value,grad_value,direction_value,criteria]
    

def Newton_line_search(x,p,alpha,function,grad_val,linear_first_derivative,linear_second_derivative,condition_check,condition_param,in_param,gamma_param):
    next_alpha=alpha
    condition=condition_check(condition_param,function,grad_val,x,p,next_alpha,in_param,gamma_param)
    while(condition==False):
        current_alpha=next_alpha
        first_deri=linear_first_derivative(current_alpha,x,p,in_param,gamma_param)
        second_deri=linear_second_derivative(current_alpha,x,p,in_param,gamma_param)
        next_alpha=current_alpha-(first_deri/second_deri)
        condition=condition_check(condition_param,function,grad_val,x,p,next_alpha,in_param,gamma_param)
        
    return next_alpha

#def secant_line_search(x,p,alpha,function,linear_first_derivative,linear_second_derivative,condition_check,condition_param):
#    condition=False
#    next_alpha=alpha
#    while(condition==False):
#        current_alpha=next_alpha
#        first_deri=linear_first_derivative(current_alpha,x,p)
#        second_deri=linear_second_derivative(current_alpha,x,p)
#        next_alpha=current_alpha-(first_deri/second_deri)
#        condition=condition_check(condition_param,function,x,p,next_alpha)
#        
#    return next_alpha   

#def Wolfe_Condition(condition_param,function,x,p,next_alpha):
#    pass

def Armijo_Condition(condition_param,function,grad_val,x,p,alpha,in_param,gamma_param):
    x=np.squeeze(np.asarray(x))
    p=np.squeeze(np.asarray(p))
    grad_val=np.squeeze(np.asarray(grad_val))
    LHS=function(x+alpha*p,in_param,gamma_param)
    RHS=function(x,in_param,gamma_param)+(condition_param*alpha*np.dot(p,grad_val))
    if LHS<=RHS:
        return True
    else:
        return False
    

In [25]:
def gaussian_pdf(inp,mean,variance):
    return (1/(2*np.pi*np.sqrt(variance)))*np.exp(-1*((inp-mean)**2/(2*variance)))

def marginal_log_likelihood_funtion_x(x,theta):
    theta=np.squeeze(np.asarray(theta))
    x=np.squeeze(np.asarray(x))
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=theta[0]
        mu2=theta[2]
        v1=theta[1]
        v2=theta[3]
        P1=theta[4]
        P2=1-theta[4]
        gauss1=gaussian_pdf(x,mu1,v1)
        gauss2=gaussian_pdf(x,mu2,v2)
        return np.sum(np.log(P1*gauss1+P2*gauss2))

def Expection_function(x,theta):
    theta=np.squeeze(np.asarray(theta))
    x=np.squeeze(np.asarray(x))
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=theta[0]
        mu2=theta[2]
        v1=theta[1]
        v2=theta[3]
        P=[theta[4],1-theta[4]]
        gauss=[gaussian_pdf(x,mu1,v1),gaussian_pdf(x,mu2,v2)]
        denominator=P[0]*gauss[0]+P[1]*gauss[1]
        return [P[0]*gauss[0]/denominator,P[1]*gauss[1]/denominator]

def Q_function(theta,x,gamma):
    theta=np.squeeze(np.asarray(theta))
    x=np.squeeze(np.asarray(x))
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=theta[0]
        mu2=theta[2]
        v1=theta[1]
        v2=theta[3]
        P1=theta[4]
        P2=1-theta[4]
        gauss1=gaussian_pdf(x,mu1,v1)
        gauss2=gaussian_pdf(x,mu2,v2)
        Q=np.log(P1*gauss1)*gamma[0]+np.log(P2*gauss2)*gamma[1]
        Q=-1*np.sum(Q)
        return Q
        
def grad_Q_function(theta,x,gamma):
    theta=np.squeeze(np.asarray(theta))
    x=np.squeeze(np.asarray(x))
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=theta[0]
        mu2=theta[2]
        v1=theta[1]
        v2=theta[3]
        P1=theta[4]
        P2=1-theta[4]
        dq_dmu1=-1*np.sum(((x-mu1)/v1)*gamma[0])
        dq_dv1=-1*np.sum(((-0.5/v1)+(0.5*(x-mu1)**2/v1**2))*gamma[0])
        dq_dmu2=-1*np.sum(((x-mu2)/v2)*gamma[1])
        dq_dv2=-1*np.sum(((-0.5/v2)+(0.5*(x-mu2)**2/v2**2))*gamma[1])
        dq_dp=-1*np.sum((1/P1)*gamma[0]-(1/P2)*gamma[1])
        return np.array([dq_dmu1,dq_dv1,dq_dmu2,dq_dv2,dq_dp])

def hessian_Q_function(theta,x,gamma):
    theta=np.squeeze(np.asarray(theta))
    x=np.squeeze(np.asarray(x))
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=theta[0]
        mu2=theta[2]
        v1=theta[1]
        v2=theta[3]
        P1=theta[4]
        P2=1-theta[4]
        d2q_dmu1=-1*np.sum(((-1/v1)*gamma[0]))
        d2q_dmu1v1=-1*np.sum((-1*(x-mu1)/v1**2)*gamma[0])
        d2q_dv1=-1*np.sum(((0.5/v1**2)+(-1*(x-mu1)**2/v1**3))*gamma[0])
        d2q_dmu2=-1*np.sum((-1/v2)*gamma[1])
        d2q_dmu2v2=-1*np.sum((-1*(x-mu2)/v2**2)*gamma[1])
        d2q_dv2=-1*np.sum(((0.5/v2**2)+(-1*(x-mu2)**2/v2**3))*gamma[1])
        d2q_dp=-1*np.sum((-1/P1**2)*gamma[0]-(-1/P2**2)*gamma[1])
        return np.array([[d2q_dmu1,d2q_dmu1v1,0,0,0],[d2q_dmu1v1,d2q_dv1,0,0,0],[0,0,d2q_dmu2,d2q_dmu2v2,0],[0,0,d2q_dmu2v2,d2q_dv2,0],[0,0,0,0,d2q_dp]])

def linearfd_Q_func_alpha(alpha,theta,theta_p,x,gamma_param):
    theta=np.squeeze(np.asarray(theta))
    theta_p=np.squeeze(np.asarray(theta_p))
    nt=theta+alpha*theta_p
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=nt[0]
        mu2=nt[2]
        v1=nt[1]
        v2=nt[3]
        P1=nt[4]
        P2=1-P1
        rval=((theta_p[4]/P1)+(0.5*(theta_p[1]/v1))+(0.5*(v1*2*(x-mu1)*-1*theta_p[0]-(x-mu1)**2*theta_p[1])/v1**2))*gamma_param[0]
        rval+=((-1*theta_p[4]/P2)+(0.5*(theta_p[3]/v2))+(0.5*(v2*2*(x-mu2)*-1*theta_p[2]-(x-mu2)**2*theta_p[3])/v2**2))*gamma_param[1]
        rval=-1*np.sum(rval)
        return rval
        
def linearsd_Q_func_alpha(alpha,theta,theta_p,x,gamma_param):
    theta=np.squeeze(np.asarray(theta))
    theta_p=np.squeeze(np.asarray(theta_p))
    nt=theta+alpha*theta_p
    if len(theta)!=5:
        sys.exit("fuction1 can handle only vector of size 5")
    else:
        mu1=nt[0]
        mu2=nt[2]
        v1=nt[1]
        v2=nt[3]
        P1=nt[4]
        P2=1-P1
        n1=v1*2*(x-mu1)*-1*theta_p[0]-(x-mu1)**2*theta_p[1]
        n2=v2*2*(x-mu2)*-1*theta_p[2]-(x-mu2)**2*theta_p[3]
        dn1=(v1*2*theta_p[0]**2)
        dn2=(v2*2*theta_p[2]**2)
        rval=((-1*theta_p[4]**2/P1**2)+(0.5*(-1*theta_p[1]**2/v1**2))+(-1*(v1**2*dn1-n1*2*v1*theta_p[1])/v1**3))*gamma_param[0]
        rval+=((theta_p[4]**2/P2**2)+(0.5*(-1*theta_p[3]**2/v2**2))+(-1*(v2**2*dn2-n2*2*v2*theta_p[3])/v2**3))*gamma_param[1]
        rval=-1*np.sum(rval)
        return rval
    


In [26]:
def E_M_Algorithm(file_name,seed_theta):
    x=np.loadtxt(file_name,dtype=int,delimiter='\n')
    optimize=unconstrained_optimizer()
    seed_theta=np.array(seed_theta)
    inital_p_x_given_theta=marginal_log_likelihood_funtion_x(x,seed_theta)
    
    if len(seed_theta)!=5:
        sys.exit("E_M can handle only vector of size 5")
   
    for i in range(100):
        # Expectation step
        gamma=Expection_function(x,seed_theta)
        
        # Maximization Step/ minimization step
            
            
        [new_theta,graval,alval,crit]=optimize.Steepest_Gradient_Descent_Method(Q_function,grad_Q_function,
                                                                         Newton_line_search,linearfd_Q_func_alpha,
                                                                         Armijo_Condition,
                                                                          1e-5,1000,seed_theta,linearsd_Q_func_alpha,
                                                                          1e-4,1e-3,x,gamma)
        
        
        new_p_x_given_theta=marginal_log_likelihood_funtion_x(x,new_theta[-1])
        seed_theta=new_theta[-1]
        c=np.sqrt((new_p_x_given_theta-inital_p_x_given_theta)**2)
        if(c<=1e-5):
            print("Convergence of Marginal probability has happened")
            return seed_theta
        else:
            inital_p_x_given_theta=new_p_x_given_theta
    
    print("Convergence of Marginal probability has not happened")
    return seed_theta
            
        
        
        
    
theta=E_M_Algorithm("image_samples.txt",[120.3091,185.1229,201.7558,201.440,0.61739])
#theta=E_M_Algorithm("image_samples.txt",[100,50,150,50,0.5])
print("Mean1:")
print(theta[0])
print("Var1:")
print(theta[1])
print("Mean2:")
print(theta[2])
print("Var2:")
print(theta[3])
print("P:")
print(theta[4])


Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Criteria of epsilon not met
Maximum iteration reached.Cr